In [1]:
import pandas as pd
pd.set_option('float_format', '{:f}'.format)

In [2]:
cdr = pd.read_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/dados_18.11/cdr/cdr_regiao_imediata_sjdr.csv', sep=';', header=None)
cdr.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,2013-03-21,11:51:30,0.370000,32,5B5F2C071D12AF13219DF5EBE05132AF,32,9FB3B96B6D5E16C9DD564AA3E84F1954,50141,nan,117,LOCAL,OI,OI,PRE PAGO,300000533
1,2013-03-21,21:03:30,0.130000,61,B3299B0E587D7275E3E4D530E9EECF50,98,6432F1DF21BA38368D9A165C739EEBB3,51601,nan,195,LONGA DISTANCIA,BRT,OI,PRE PAGO,300000210
2,2013-03-21,17:33:20,0.330000,61,85D5C50A6D882CA8E4BB00BCA3574417,61,0D8583F810B9720A8032BB939F12B3FF,51601,nan,117,LOCAL,BRT,BRT,PRE PAGO,300000210
3,2013-03-21,11:58:26,1.270000,61,6C10A9E9F325CAA3CCB7F9A0D6983D2A,61,B0C50ED1DEC9E06E4C64E7419DDC4B09,51601,nan,117,LOCAL,BRT,BRT,PRE PAGO,300000210
4,2013-03-21,16:06:59,0.170000,61,29122DCF4189448865B03F24C2DFEAE6,61,E0B097D86BF9C02A23011610EBAF47FD,51601,nan,77,LOCAL,BRT,GVT,PRE PAGO,300000210


In [3]:
cdr.rename(columns={0:'DATE', 1:'HOUR', 2:'DURATION', 3:'DDD_USER_FROM',
                    4:'USER_FROM', 5:'DDD_USER_TO', 6:'USER_TO', 7: 'ANTENNA'},inplace=True)
cdr = cdr.drop(8,1)

# Análise Exploratória

Parte 1: Análise sobre os usuários

In [4]:
#USER_FROM Distintos
len(cdr['USER_FROM'].unique())

445162

In [5]:
#USER_TO Distintos
len(cdr['USER_TO'].unique())

1198467

In [6]:
#Id's distintos em USER_FROM e USER_TO
len(pd.unique(cdr[['USER_FROM', 'USER_TO']].values.ravel('k')))

1303639

Parte 2: Análise sobre a duração das ligações

In [7]:
cdr['DURATION'].describe()

count   9293316.000000
mean          1.496004
std           8.311616
min           0.000000
25%           0.220000
50%           0.600000
75%           1.420000
max        5758.000000
Name: DURATION, dtype: float64

In [8]:
cdr['DURATION'].median()

0.6

# Filtros da base

Selecionando apenas registros onde a duração da ligação está entre 0.07 e 120 minutos

In [9]:
cdr_new = cdr[cdr['DURATION'].between(0.07,120)]

In [10]:
#Mesmo com o filtro de duração, manteve-se pouco mais de 99% dos registros da base original
len(cdr_new)/len(cdr)

0.9934378643747829

Filtrando a partir do número de ligações

In [11]:
#Selecionando os usuários com o número de ligações entre 3 e 500
cdr_filter_by_quantity_of_call = cdr_new.groupby('USER_FROM').size().reset_index().rename(columns={0:'QUANTITY_OF_CALLS'})

cdr_filter_by_quantity_of_call = cdr_filter_by_quantity_of_call.loc[(
                                (cdr_filter_by_quantity_of_call['QUANTITY_OF_CALLS'] >= 3) 
                                & (cdr_filter_by_quantity_of_call['QUANTITY_OF_CALLS'] <= 500))]

cdr_filter_by_quantity_of_call.head()

,USER_FROM,QUANTITY_OF_CALLS
1,000008B27F635FC0AFF9BAC21880B796,4
3,00002BC8FEAE1CB1EC0E1A8AF160B16E,12
4,00003F26576D1122848B6F3CEEC250E5,12
7,0000FFB82D6062A83F28FEE4D0BD5E4D,3
8,00014FE9BDE3EE54C61AB62700A1A2AF,14


In [12]:
#Selecionando os usuários com pelo menos 3 ligações em dias distintos
cdr_filter_by_quantity_of_days = cdr_new.groupby('USER_FROM').DATE.nunique().to_frame('QUANTITY_OF_DAYS').reset_index()
cdr_filter_by_quantity_of_days = cdr_filter_by_quantity_of_days.loc[(cdr_filter_by_quantity_of_days['QUANTITY_OF_DAYS'] >= 3)]
cdr_filter_by_quantity_of_days.head()

,USER_FROM,QUANTITY_OF_DAYS
1,000008B27F635FC0AFF9BAC21880B796,3
3,00002BC8FEAE1CB1EC0E1A8AF160B16E,6
4,00003F26576D1122848B6F3CEEC250E5,9
7,0000FFB82D6062A83F28FEE4D0BD5E4D,3
8,00014FE9BDE3EE54C61AB62700A1A2AF,5


In [13]:
#Interseção entre os dois dataframes (de ligações e dias com ligações)
df_filter_over_call = pd.merge(cdr_filter_by_quantity_of_call, cdr_filter_by_quantity_of_days,
                               how='inner',on=['USER_FROM'])

df_filter_over_call.head()

,USER_FROM,QUANTITY_OF_CALLS,QUANTITY_OF_DAYS
0,000008B27F635FC0AFF9BAC21880B796,4,3
1,00002BC8FEAE1CB1EC0E1A8AF160B16E,12,6
2,00003F26576D1122848B6F3CEEC250E5,12,9
3,0000FFB82D6062A83F28FEE4D0BD5E4D,3,3
4,00014FE9BDE3EE54C61AB62700A1A2AF,14,5


In [14]:
#Filtrando cdr_new: somente os USER_FROM que constam em df_filter_over_call continuaram na análise
cdr_after_filters_of_calls = cdr_new[(cdr_new['USER_FROM'].isin(df_filter_over_call['USER_FROM']))]
cdr_after_filters_of_calls.head()

,DATE,HOUR,DURATION,DDD_USER_FROM,USER_FROM,DDD_USER_TO,USER_TO,ANTENNA,9,10,11,12,13,14
0,2013-03-21,11:51:30,0.370000,32,5B5F2C071D12AF13219DF5EBE05132AF,32,9FB3B96B6D5E16C9DD564AA3E84F1954,50141,117,LOCAL,OI,OI,PRE PAGO,300000533
1,2013-03-21,21:03:30,0.130000,61,B3299B0E587D7275E3E4D530E9EECF50,98,6432F1DF21BA38368D9A165C739EEBB3,51601,195,LONGA DISTANCIA,BRT,OI,PRE PAGO,300000210
2,2013-03-21,17:33:20,0.330000,61,85D5C50A6D882CA8E4BB00BCA3574417,61,0D8583F810B9720A8032BB939F12B3FF,51601,117,LOCAL,BRT,BRT,PRE PAGO,300000210
3,2013-03-21,11:58:26,1.270000,61,6C10A9E9F325CAA3CCB7F9A0D6983D2A,61,B0C50ED1DEC9E06E4C64E7419DDC4B09,51601,117,LOCAL,BRT,BRT,PRE PAGO,300000210
4,2013-03-21,16:06:59,0.170000,61,29122DCF4189448865B03F24C2DFEAE6,61,E0B097D86BF9C02A23011610EBAF47FD,51601,77,LOCAL,BRT,GVT,PRE PAGO,300000210


Selecionando os registros onde as ligações ocorreram entre 19:00 e 06:00 ou tenham sido realizadas em qualquer horário do domingo

In [15]:
cdr_after_filters_of_calls['DATE'] = pd.to_datetime(cdr_after_filters_of_calls['DATE'])
cdr_after_filters_of_calls['DAY_OF_WEEK'] = cdr_after_filters_of_calls['DATE'].dt.day_name()

/home/gustavo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/gustavo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
cdr_after_filters_of_calls.sample(5)
cdr_after_filters_of_calls.to_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/dados_18.11/cdr_after_filtes.csv')

In [17]:
#selecionando as ligações que ocorreram entre 19h e 6h ou no domingo em qualquer horário
calls_night_or_sunday = cdr_after_filters_of_calls[(cdr_after_filters_of_calls.HOUR.between('19:00:00','23:59:00')) 
                               | (cdr_after_filters_of_calls.HOUR.between('00:00:00','06:00:00')) 
                               | (cdr_after_filters_of_calls['DAY_OF_WEEK'] == 'Sunday')]

calls_night_or_sunday

,DATE,HOUR,DURATION,DDD_USER_FROM,USER_FROM,DDD_USER_TO,USER_TO,ANTENNA,9,10,11,12,13,14,DAY_OF_WEEK
1,2013-03-21,21:03:30,0.130000,61,B3299B0E587D7275E3E4D530E9EECF50,98,6432F1DF21BA38368D9A165C739EEBB3,51601,195,LONGA DISTANCIA,BRT,OI,PRE PAGO,300000210,Thursday
6,2013-03-21,20:30:29,7.020000,61,BA4DEA1F710C1216F74F1255150C49D6,61,4F6DCBA36292B8487743E13319E67DF6,51601,77,LOCAL,BRT,GVT,PRE PAGO,300000210,Thursday
9,2013-03-21,01:10:47,0.830000,32,1AC8490BA99709F652B4A5C3525D3C82,32,B6BB8A38B286A41978E732BDE0249DB5,50141,117,LOCAL,OI,OI,POS PAGO,300000188,Thursday
10,2013-03-21,21:11:55,6.870000,32,05AC98D91ED3C2821F53519B19CCBC58,32,A3ACD65D308B86F4487DB35A5F2E50EB,50141,117,LOCAL,OI,OI,PRE PAGO,300000533,Thursday
13,2013-03-21,20:29:21,0.130000,61,BA4DEA1F710C1216F74F1255150C49D6,21,A49BA005920B8BA24BEE40F455B69B9B,51601,117,LOCAL,BRT,OI,PRE PAGO,300000210,Thursday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9293309,2013-04-19,20:57:51,0.350000,82,D45A858A68F7839B9866724A878618E3,82,D9524BFBE34B8E548798414B925FCF78,23441,117,LOCAL,OI,OI,PRE PAGO,300000210,Friday
9293310,2013-04-19,22:42:53,4.300000,82,876169C4D43647A649D53D6AEBBCBACE,82,A8890648CEF226453D4B76E534F99466,23451,117,LOCAL,OI,OI,PRE PAGO,300000210,Friday
9293313,2013-04-19,22:29:57,6.280000,82,06D5FF5901283A69AD23C295D3E24EFB,82,1029D7F81B2BCE9812F20CBE5A01599E,23451,117,LOCAL,OI,OI,PRE PAGO,300000533,Friday
9293314,2013-04-19,01:56:58,1.580000,82,7670BF3A1F8C74B01779DDA73554BC8D,82,94253ECA00D72C7FBF8C6706192CF3F8,23451,117,LOCAL,OI,OI,PRE PAGO,300000210,Friday


In [ ]:
calls_night_or_sunday.to_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/dados_18.11/calls_night_or_sunday.csv')

# Agrupando as ligações por dia da semana

In [ ]:
cdr_monday = cdr_after_filters_of_calls.loc[cdr_after_filters_of_calls['DAY_OF_WEEK'] == 'Monday']
cdr_monday.to_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/CDR_by_day_of_week/cdr_monday.csv')

In [ ]:
cdr_tuesday = cdr_after_filters_of_calls.loc[cdr_after_filters_of_calls['DAY_OF_WEEK'] == 'Tuesday']
cdr_tuesday.to_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/CDR_by_day_of_week/cdr_tuesday.csv')

In [ ]:
cdr_wednesday = cdr_after_filters_of_calls.loc[cdr_after_filters_of_calls['DAY_OF_WEEK'] == 'Wednesday']
cdr_wednesday.to_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/CDR_by_day_of_week/cdr_wednesday.csv')

In [ ]:
cdr_thursday = cdr_after_filters_of_calls.loc[cdr_after_filters_of_calls['DAY_OF_WEEK'] == 'Thursday']
cdr_thursday.to_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/CDR_by_day_of_week/cdr_thursday.csv')

In [ ]:
cdr_friday = cdr_after_filters_of_calls.loc[cdr_after_filters_of_calls['DAY_OF_WEEK'] == 'Friday']
cdr_friday.to_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/CDR_by_day_of_week/cdr_friday.csv')

In [ ]:
cdr_saturday = cdr_after_filters_of_calls.loc[cdr_after_filters_of_calls['DAY_OF_WEEK'] == 'Saturday']
cdr_saturday.to_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/CDR_by_day_of_week/cdr_saturday.csv')

In [ ]:
cdr_sunday = cdr_after_filters_of_calls.loc[cdr_after_filters_of_calls['DAY_OF_WEEK'] == 'Sunday']
cdr_sunday.to_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/CDR_by_day_of_week/cdr_sunday.csv')